# Trekpedia


Writing a web-scraper to pull all `Star Trek(tm)` series data from Wikipedia.

## Stage 2 - get episode data
Create separate json files containing episode data for each Series.
For now we will keep all seasons in one file but may break this into individual ones depending on how much data we finally grab.

In [1]:
# common setup...
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

# don't truncate Pandas.DataFrame cell contents when displaying.
# pd.set_option('display.max_colwidth', None)

In [2]:
# read in the data from stage 1 ...
df = pd.read_json('../output/star_trek_series_info.json', orient='index')
# df

In [3]:
# create a subset that only contains the seasons we want to work on.
# df2 = pd.DataFrame(df.iloc[6]).transpose()
df2 = df
df2

,name,url,season_count,episode_count,episodes_url,dates,logo
1,The Original Series,https://en.wikipedia.org/wiki/Star_Trek:_The_O...,3,79,https://en.wikipedia.org/wiki/List_of_Star_Tre...,"September 8, 1966 - June 3, 1969",https://upload.wikimedia.org/wikipedia/commons...
2,The Animated Series,https://en.wikipedia.org/wiki/Star_Trek:_The_A...,2,22,https://en.wikipedia.org/wiki/Star_Trek:_The_A...,"September 8, 1973 - October 12, 1974",https://upload.wikimedia.org/wikipedia/commons...
3,The Next Generation,https://en.wikipedia.org/wiki/Star_Trek:_The_N...,7,178,https://en.wikipedia.org/wiki/List_of_Star_Tre...,"September 28, 1987 - May 23, 1994",https://upload.wikimedia.org/wikipedia/commons...
4,Deep Space Nine,https://en.wikipedia.org/wiki/Star_Trek:_Deep_...,7,176,https://en.wikipedia.org/wiki/List_of_Star_Tre...,"January 4, 1993 - May 31, 1999",https://upload.wikimedia.org/wikipedia/commons...
5,Voyager,https://en.wikipedia.org/wiki/Star_Trek:_Voyager,7,172,https://en.wikipedia.org/wiki/List_of_Star_Tre...,"January 16, 1995 - May 23, 2001",https://upload.wikimedia.org/wikipedia/commons...
6,Enterprise,https://en.wikipedia.org/wiki/Star_Trek:_Enter...,4,98,https://en.wikipedia.org/wiki/List_of_Star_Tre...,"September 26, 2001 - May 13, 2005",https://upload.wikimedia.org/wikipedia/commons...
7,Discovery,https://en.wikipedia.org/wiki/Star_Trek:_Disco...,4,55,https://en.wikipedia.org/wiki/List_of_Star_Tre...,"September 24, 2017 - present",https://upload.wikimedia.org/wikipedia/commons...
8,Short Treks,https://en.wikipedia.org/wiki/Star_Trek:_Short...,2,10,https://en.wikipedia.org/wiki/Star_Trek:_Short...,"October 4, 2018 - January 9, 2020",
9,Picard,https://en.wikipedia.org/wiki/Star_Trek:_Picard,2,20,https://en.wikipedia.org/wiki/Star_Trek:_Picard,"January 23, 2020 - present",https://upload.wikimedia.org/wikipedia/commons...
10,Lower Decks,https://en.wikipedia.org/wiki/Star_Trek:_Lower...,2,20,https://en.wikipedia.org/wiki/Star_Trek:_Lower...,"August 6, 2020 - present",https://upload.wikimedia.org/wikipedia/commons...


In [4]:
# helper function to clean up strings - remove unicode and any brackets
def clean_string(str, underscores=False, brackets=False, lowercase=False):
    print(str)
    if underscores:
        str = str.replace(" ", "_").replace(".", "_").replace("__", "_")
    if brackets:
        str = "".join(re.split("\(|\)|\[|\]", str)[::2])
    if lowercase:
        str = str.lower()
    return ' '.join(str.split())

In [5]:
FILE_TEMPLATE = '../output/star_trek_series_{}_{}_episodes.json'

for i, row in enumerate(df2.itertuples(),1):  
    
    if row.name in ['Prodigy']:
        continue

    print(f'Processing : {row.name}')
    filename = FILE_TEMPLATE.format(i,row.name.replace(" ", "_").lower())
    print(f"  -> Using URL : {row.episodes_url}")
    print(f"  -> Storing episodes to '{filename}'")
    
    season_final = dict()
    season_all = dict()
    
    # get and parse the webpage...
    result = requests.get(row.episodes_url)
    bs = BeautifulSoup(result.text, 'lxml')

    # wrap all this in a Try:Except block, there are a few series which need special handling...
    try:
        # find the episode summary table, will be the first table with the below classes in the document
        summary_table = bs.find('table', attrs={'class': 'wikitable plainrowheaders'})

        if summary_table:
            summary_rows = summary_table.find('tbody').find_all('tr')[2:]
        else:
            print("   x No Summary Table found, currently skipping this Series ...")
            continue
        
        
        for season in summary_rows:
            season_data = dict()
            link = season.find('th')
            cells = season.find_all('td')         
            
            try:
                season_number = int(link.text)
            except AttributeError:
                continue
            
            # exit the loop if we have processed the actual number of seasons. Usually this is not needed, 
            # however it is for the new series that are still in progress. 
            if season_number > row.season_count:
                break
            
            print(f'  -> Processing season: {season_number} of {row.season_count}')
            season_id = link.a['href'][1:]
            season_data['total'] = clean_string(cells[0].text, brackets=True)
            # get start/end data and remove unicode chars. 
            # Still need to remove the date in backets at the end of each
            season_data['season_start'] = clean_string(" ".join(cells[1].text.split()), brackets=True)
            season_data['season_end'] = clean_string(" ".join(cells[2].text.split()), brackets=True)
            season_data['episodes'] = list()

            # now get the actual episodes for this season...
            section = bs.find('span', id=season_id)
            table = section.findNext('table').find('tbody').find_all('tr')
            
            # split the headers out into a list, as they change between series and even seasons!
            # at this time we also remove any unicode stuff 
            h = table[0].find_all('th')        
            headers = [clean_string(x.text, underscores=True, brackets=True, lowercase=True) for x in h]
            # remove the overall count as this is a TH not a TD and will skew the indexing later...
            headers.remove('no_overall')
            
            # 'episodes' will consist of one row for each episode, except ds9 and voy who also put summary
            # after each one and confuse things!
            episodes = table[1:]
        
            episode_list = list()
            # loop over each episode, getting the relevant data. We may grab more info in the future.
            for episode in episodes:
                episode_data = dict()
                # protect the next operation - if the th is not found (ie tas, ds9, voy) just skip over this 
                # one as it is a summary...
                try:
                    episode_data['num_overall'] = clean_string(episode.find('th').text, brackets=True)
                except AttributeError:
                    continue
                cells = episode.find_all('td')
                episode_data['num_in_season'] = cells[headers.index('no_inseason')].text
                
                # need to do some tweaking, sometimes the first episode is in 2 parts
                # need to detect this and split them. Alternative is to have a hard-coded list, as it
                # happens very rarely.
                
                # get the required data using the header indexes, otherwise will mess up on ds9-s4 and later
                # since they add new columns to the table.
                episode_data['title'] = clean_string(cells[headers.index('title')].text.replace('"',''), brackets=True)
                try:
                    # put these in try/except as some don't have episode links
                    
                    link_url = cells[headers.index('title')].a['href']
                    if "cite_note" in link_url:
                        raise TypeError() 
                    episode_data['link'] = f"https://en.wikipedia.org{link_url}"
                except TypeError:
                    # set the link url to an empty string...
                    episode_data['link'] = ''
                
                episode_data['director'] = clean_string(cells[headers.index('directed_by')].text, brackets=True)

                # air date needs a regex as is listed differently in later series...
                airdate_idx = [i for i, item in enumerate(headers) if re.search('^original.*date$', item)][0]
                episode_data['air_date'] = clean_string(cells[airdate_idx].text, brackets=True)
                
                episode_list.append(episode_data)
                
            # consolidate into a format suitable for writing to JSON
            season_data['episodes'] = episode_list
            season_all[season_number] = season_data
            season_final['seasons'] = season_all
    except AttributeError as e:
        print(f"  => ERROR, need to investigate! ({e}) at line number: {e.__traceback__.tb_lineno}")
    finally:
       # write to json file...
        with open (filename, 'w', encoding='utf-8') as f:
            json.dump(season_final, f, ensure_ascii=False, indent=4)
            print(f"  -> Done.\n")


Processing : The Original Series
  -> Using URL : https://en.wikipedia.org/wiki/List_of_Star_Trek:_The_Original_Series_episodes
  -> Storing episodes to '../output/star_trek_series_1_the_original_series_episodes.json'
  -> Processing season: 1 of 3
29
September 8, 1966 (1966-09-08)
April 13, 1967 (1967-04-13)
No.overall
No. inseason
Title
Directed by
Written by
Original air date [2]
Prod.code
1
The Man Trap
Marc Daniels
September 8, 1966 (1966-09-08)
2
Charlie X
Lawrence Dobkin
September 15, 1966 (1966-09-15)
3
Where No Man Has Gone Before
James Goldstone
September 22, 1966 (1966-09-22)
4
The Naked Time
Marc Daniels
September 29, 1966 (1966-09-29)
5
The Enemy Within
Leo Penn
October 6, 1966 (1966-10-06)
6
Mudd's Women
Harvey Hart
October 13, 1966 (1966-10-13)
7
What Are Little Girls Made Of?
James Goldstone
October 20, 1966 (1966-10-20)
8
Miri
Vincent McEveety
October 27, 1966 (1966-10-27)
9
Dagger of the Mind
Vincent McEveety
November 3, 1966 (1966-11-03)
10
The Corbomite Maneuver
Jos

## Current Bugs
1. Some 2-part episodes have bad season and overall number due to table layout.
2. [`ALL FIXED`] At least DS9 from season 4 and Voyager, Enterprise add a
   'stardate' column which messes up the column count and therefore the
   'Original Air Date' field. Voyager also adds 'featured character' to this
   confusion. Later the air date field is renamed too.
3. [`FIXED`] Discovery errors out after first season
4. [`ALL FIXED`] From Short Treks to Lower Decks error out on line 76, more
   formatting changes.
5. Prodigy errors out at the start, this is because it has no Season Summary
   table. We may want to change the way we get the data, using the
   'wikiepisodetable' class directly. For now I'm skipping this as it prob needs
   a rewrite of the base logic.

# Different Method test.
Leaving this in as markdown for future reference, but the read_html function of Pandas is not really working well for these tables.

```python
FILE_TEMPLATE = 'output/star_trek_series_{}_{}_episodes.json'

for row in df.itertuples(index=True):
   
    print(f'Processing : {row.name}')
    filename = FILE_TEMPLATE.format(row.Index,row.name.replace(" ", "_").lower())
    print(f"  -> Using URL : {row.episodes_url}")
    print(f"  -> Storing episodes to '{filename}'")
    
    season_final = dict()
    season_all = dict()
    
    # get and parse the webpage...
    result = requests.get(row.episodes_url)
    bs = BeautifulSoup(result.text, 'lxml')
    
    # wrap all this in a Try:Except block, there are a few series which need special handling...
    try:
        # find the episode summary table, will be the first table with the below classes in the document
        summary_table = bs.find('table', attrs={'class': 'wikitable plainrowheaders'})
        
        summary_rows = summary_table.find('tbody').find_all('tr')[2:]
        
        for season in summary_rows:
            season_data = dict()
            
            link = season.find('th')
            cells = season.find_all('td')         
            
            season_number = link.text
            season_id = link.a['href'][1:]
            season_data['total'] = cells[1].text
            # get start/end data and remove unicode chars. 
            # Still need to remove the date in backets at the end of each
            season_data['start'] = " ".join(cells[2].text.split())
            season_data['end'] = " ".join(cells[3].text.split())
            season_data['episodes'] = list()
            
            # now get the actual episodes for this season...
            section = bs.find('span', id=season_id)
            table = section.findNext('table')
            
            table_data = pd.read_html(str(table), parse_dates=True)
            print(table_data)
            print("[>----------------<]")
            
    except AttributeError as e:
        print(f"  => Error, need to investigate! ({e}) at line number: {e.__traceback__.tb_lineno}")
```